In [1]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist


# Load the MNIST dataset
(train_images, train_labels), _ = mnist.load_data()

2024-05-24 12:02:50.062451: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 12:02:50.062539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 12:02:50.217690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Example to explain the contours and chain code extracted from one image

In [2]:
image= train_images[0].reshape(28,28,1)

ret,thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY )
contours,_= cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

contours = np.array(contours[0])
boundary = contours.reshape(contours.shape[0],2)

print('coordinates =  ',boundary)


chaincode=[]
k=0
while k<len(boundary)-1:
    x,z=boundary[k]
    l=k+1
    next_x,next_z=boundary[l]
    if(x==next_x and z+1==next_z):
        code=0
    elif(x-1==next_x and z+1==next_z):
        code=1
    elif(x-1==next_x and z==next_z):
        code=2
    elif(x-1==next_x and z-1==next_z):
        code=3
    elif(x==next_x and z-1==next_z):
        code=4
    elif(x+1==next_x and z-1==next_z):
        code=5
    elif(x+1==next_x and z==next_z):
        code=6
    elif(x+1==next_x and z+1==next_z):
        code=7
    k=k+1
    chaincode.append(code)

print("chain =  ",chaincode)

coordinates =   [[17  5]
 [16  6]
 [15  6]
 [14  6]
 [13  6]
 [12  6]
 [11  6]
 [10  7]
 [ 9  7]
 [ 8  7]
 [ 8  8]
 [ 9  9]
 [10  8]
 [11  9]
 [11 10]
 [11 11]
 [12 12]
 [13 13]
 [14 14]
 [15 15]
 [16 15]
 [17 16]
 [17 17]
 [16 18]
 [15 18]
 [14 19]
 [13 19]
 [12 20]
 [11 21]
 [10 21]
 [ 9 22]
 [ 8 22]
 [ 7 22]
 [ 6 23]
 [ 5 23]
 [ 4 24]
 [ 5 24]
 [ 6 24]
 [ 7 24]
 [ 8 24]
 [ 9 24]
 [10 24]
 [11 23]
 [12 23]
 [13 22]
 [14 21]
 [15 21]
 [16 20]
 [17 20]
 [18 19]
 [19 19]
 [19 18]
 [19 17]
 [19 16]
 [18 15]
 [17 15]
 [16 14]
 [15 13]
 [14 13]
 [13 12]
 [13 11]
 [12 10]
 [13  9]
 [14  8]
 [15  8]
 [16  8]
 [17  9]
 [17  8]
 [17  7]
 [18  6]
 [19  6]
 [20  6]
 [21  6]
 [22  6]
 [22  5]
 [21  5]
 [20  5]
 [19  6]
 [18  5]]
chain =   [1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 7, 5, 7, 0, 0, 7, 7, 7, 7, 6, 7, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 6, 6, 6, 6, 6, 6, 5, 6, 5, 5, 6, 5, 6, 5, 6, 4, 4, 4, 3, 2, 3, 3, 2, 3, 4, 3, 5, 5, 6, 6, 7, 4, 4, 5, 6, 6, 6, 6, 4, 2, 2, 1, 3]


In [3]:
# Feature extraction function using chain code
def extract_features(image):
    # Convert image to binary
    ret, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(image=thresh, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contours = np.array(contours[0])
    boundary = contours.reshape(contours.shape[0],2)


    # Compute chain code
    chaincode = []
    k = 0
    while k < len(boundary) - 1:
        x, y = boundary[k]
        l = k + 1
        next_x, next_y = boundary[l]
        if (x == next_x and y + 1 == next_y):
            code = 0
        elif (x - 1 == next_x and y + 1 == next_y):
            code = 1
        elif (x - 1 == next_x and y == next_y):
            code = 2
        elif (x - 1 == next_x and y - 1 == next_y):
            code = 3
        elif (x == next_x and y - 1 == next_y):
            code = 4
        elif (x + 1 == next_x and y - 1 == next_y):
            code = 5
        elif (x + 1 == next_x and y == next_y):
            code = 6
        elif (x + 1 == next_x and y + 1 == next_y):
            code = 7
        k = k + 1
        chaincode.append(code)

    return chaincode

## Apply the chain code feature extraction on MNIST data set 

In [4]:
# Extract features from all images
features = [extract_features(image.reshape(28, 28, 1)) for image in train_images]

# Find the maximum length of features
max_length = max(len(feature) for feature in features)

# Pad or truncate features to the maximum length
padded_features = [feature + [0] * (max_length - len(feature)) for feature in features]

# Convert features to numpy array
X = np.array(padded_features)
y = train_labels  # Assuming train_labels contain the corresponding labels

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Using the extracted features on the SVM model

In [5]:
# Train SVM model
clf = svm.SVC(kernel= "poly")
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.901
